---
title: AdaBoost
date: 2019-09-06
categories: [人工智能, 监督学习]
mathjax: true
---

## 基本介绍

1. 初始化训练数据的权值分布。每⼀个训练样本最开始时都被赋予相同的权值：1/N
2. 进⾏多轮迭代，⽤m = 1,2, ..., M表⽰迭代的第多少轮
    - 使⽤具有权值分布Dm的训练数据集学习，得到基本分类器（选取让误差率最低的阈值来设计基本分类器）
    - 计算Gm(x)在训练数据集上的分类误差率，Gm(x)在训练数据集上的误差率em就是被Gm(x)误分类样本的权值之和
$$e_m=\sum_{N}^{i=1}{w_{mi}I(G_m(x_i)\neq y_i)}$$
    - 计算Gm(x)的系数，am表⽰Gm(x)在最终分类器中的重要程度（⽬的：得到基本分类器在最终分类器中所占的权重）：
$$\alpha_m={\frac{1}{2}}log{\frac{1-e_m}{e_m}}$$
em <= 1/2时，am >= 0，且am随着em的减⼩⽽增⼤，意味着分类误差率越⼩的基本分类器在最终分类器中的作⽤越⼤
    - 更新训练数据集的权值分布（为了得到样本的新的权值分布），⽤于下⼀轮迭代，使得被基本分类器Gm(x)误分类样本的权值增⼤，⽽被正确分类样本的权值减⼩  
3. 组合各个弱分类器，从⽽得到最终分类器
$$G(x)=sign(f(x))=sign(\sum_{m=1}^{M}\alpha_m G_m(x))$$

## AdaBoostClassifier

In [1]:
from sklearn.ensemble import AdaBoostClassifier  
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
iris_data = iris['data']
iris_label = iris['target']
iris_target_name = iris['target_names']
X = np.array(iris_data)
Y = np.array(iris_label)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

clf = AdaBoostClassifier()
clf = clf.fit(x_train, y_train)

In [4]:
from sklearn.metrics import roc_curve, auc

y_pred = clf.predict(x_test)

false_positive_rate, true_positive_rate, thresholds = roc_curve(
    y_test, y_pred, pos_label=2)

roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.9225589225589226

- base_estimator: 基分类器，默认是决策树，在该分类器基础上进行boosting，理论上可以是任意一个分类器，但是如果是其他分类器时需要指明样本权重
- n_estimators: 基分类器提升（循环）次数，默认是50次，这个值过大，模型容易过拟合；值过小，模型容易欠拟合
- learning_rate: 学习率，表示梯度收敛速度，默认为1，如果过大，容易错过最优值，如果过小，则收敛速度会很慢；该值需要和n_estimators进行一个权衡，当分类器迭代次数较少时，学习率可以小一些，当迭代次数较多时，学习率可以适当放大
- algorithm: boosting算法，也就是模型提升准则，有两种方式SAMME, 和SAMME.R两种，默认是SAMME.R，两者的区别主要是弱学习器权重的度量，前者是对样本集预测错误的概率进行划分的，后者是对样本集的预测错误的比例，即错分率进行划分的，默认是用的SAMME.R
- random_state: 随机种子设置

> 参考：

1. [从机器学习到深度学习：基于Scikit-learn与TensorFlow的高效开发实战](http://www.broadview.com.cn/book/5337)
2. [AdaBoost原理详解](https://www.cnblogs.com/ScorpioLu/p/8295990.html)
3. [AdaBoostClassifier参数](https://www.cnblogs.com/mdevelopment/p/9445090.html)